In [ ]:
pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.1 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-

"""
GERADOR DE CONTRATO DE LOCAÇÃO EM PDF - VERSÃO 4.5

Este script lê os dados de um arquivo 'dados.txt', formata as informações
para garantir a padronização (iniciais maiúsculas para nomes, etc.) e monta
um contrato de locação com base em um modelo específico, gerando um arquivo
PDF formatado com anexo de fotos.

Esta versão inclui opções interativas para escolher:
1. A modalidade detalhada de cobrança de água e energia elétrica (fixo ou %).
2. A inclusão e o valor de uma taxa de manutenção de áreas comuns.
3. O tipo de vigência do contrato (típico de 30 meses ou atípico com prazo menor).
4. A forma de pagamento do depósito caução (à vista ou parcelado).
5. O método de cálculo da multa por rescisão (proporcional ou valor fixo).

Requisitos:
- Python 3.x
- Biblioteca ReportLab (instale com: pip install reportlab)

Como usar:
1. Salve este script com o nome 'gerador_contrato.py'.
2. Na MESMA PASTA, crie um arquivo 'dados.txt' e preencha-o com as
   informações necessárias.
   - Para contratos com vigência ATÍPICA (prazo menor que 30 meses),
     inclua no 'dados.txt' a chave:
     JUSTIFICATIVA_CONTRATO_ATIPICO = [escreva aqui o motivo]
3. Na MESMA PASTA, crie um diretório chamado 'imagens'.
4. Coloque dentro da pasta 'imagens' as fotos para o Anexo I.
5. Execute o script: python gerador_contrato.py
6. Responda às perguntas que serão feitas no terminal.
7. O arquivo 'contrato_locacao.pdf' será criado na mesma pasta.
"""

import os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.colors import black

# --- 1. FUNÇÃO PARA LER E PADRONIZAR OS DADOS ---

def ler_dados_contrato(caminho_arquivo="dados.txt"):
    """
    Lê um arquivo de texto no formato CHAVE=VALOR, padroniza os dados
    e retorna um dicionário.
    """
    dados = {}

    # Listas de chaves para formatação específica
    keys_title_case = [
        'LOCADORA_NOME', 'LOCATARIO_NOME', 'TESTEMUNHA1_NOME', 'TESTEMUNHA2_NOME',
        'CIDADE_ASSINATURA', 'LOCATARIO_PROFISSAO', 'LOCADORA_ENDERECO',
        'LOCATARIO_ENDERECO_ANTERIOR', 'IMOVEL_ENDERECO'
    ]
    keys_lower_case = ['JUSTIFICATIVA_CONTRATO_ATIPICO']

    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            for linha in f:
                if '=' in linha:
                    chave, valor = linha.strip().split('=', 1)
                    chave = chave.lstrip('\ufeff').strip()
                    valor = valor.strip()

                    # Aplica a padronização com base na chave
                    if chave in keys_title_case:
                        dados[chave] = valor.title()
                    elif chave in keys_lower_case:
                        dados[chave] = valor.lower()
                    else:
                        # Mantém o valor original para as demais chaves (CPF, RG, CEPs, etc.)
                        dados[chave] = valor

    except FileNotFoundError:
        print(f"Erro: O arquivo de dados '{caminho_arquivo}' não foi encontrado.")
        print("Por favor, certifique-se de que 'dados.txt' está na mesma pasta que o script.")
        return None
    return dados

# --- 2. FUNÇÕES PARA GERAR AS PARTES DO CONTRATO ---

def gerar_qualificacao_e_intro(dados, modalidade_vigencia):
    """
    Gera a qualificação das partes e os parágrafos introdutórios,
    ajustando o prazo de vigência e a justificativa conforme a escolha.
    """
    if modalidade_vigencia == '1': # Típico
        prazo_texto = "30 (trinta) meses"
        justificativa_paragrafo = ""
    else: # Atípico
        prazo_texto = dados.get('PRAZO_VIGENCIA', '[PRAZO NÃO ENCONTRADO]')
        justificativa_texto = dados.get('JUSTIFICATIVA_CONTRATO_ATIPICO', '[JUSTIFICATIVA NÃO FORNECIDA]')
        justificativa_paragrafo = f"""
        <br/><br/>
        O presente contrato é celebrado com prazo inferior a 30 (trinta) meses em razão de se destinar a {justificativa_texto}, em conformidade com o disposto na legislação aplicável.
        """

    return f"""
    <b>NESTE ATO denominada LOCADORA:</b> {dados.get('LOCADORA_NOME', '[NOME NÃO ENCONTRADO]')}, nacionalidade brasileira, estado civil casada, CPF: {dados.get('LOCADORA_CPF', '[CPF NÃO ENCONTRADO]')}, RG: {dados.get('LOCADORA_RG', '[RG NÃO ENCONTRADO]')}, residente e domiciliada no endereço: {dados.get('LOCADORA_ENDERECO', '[ENDEREÇO NÃO ENCONTRADO]')}. CEP: {dados.get('LOCADORA_CEP', '[CEP NÃO ENCONTRADO]')}.
    <br/><br/>
    <b>OUTRO LADO, denominado(a) LOCATÁRIO(A):</b> {dados.get('LOCATARIO_NOME', '[NOME NÃO ENCONTRADO]')} RG: {dados.get('LOCATARIO_RG', '[RG NÃO ENCONTRADO]')} nacionalidade BRASILEIRA, CPF: {dados.get('LOCATARIO_CPF', '[CPF NÃO ENCONTRADO]')}. Profissão: {dados.get('LOCATARIO_PROFISSAO', '[PROFISSÃO NÃO ENCONTRADA]')}. Endereço anterior: {dados.get('LOCATARIO_ENDERECO_ANTERIOR', '[ENDEREÇO ANTERIOR NÃO ENCONTRADO]')}.
    <br/><br/>
    O imóvel de propriedade da LOCADORA situa-se no endereço: {dados.get('IMOVEL_ENDERECO', '[ENDEREÇO NÃO ENCONTRADO]')}, CEP: {dados.get('IMOVEL_CEP', '[CEP NÃO ENCONTRADO]')}. O prazo de locação do imóvel mencionado acima terá validade de <b>{prazo_texto}</b>, iniciando a partir da data de assinatura deste contrato.
    {justificativa_paragrafo}
    <br/><br/>
    A presente LOCAÇÃO destina-se ao uso do imóvel somente pra fins residenciais, ficando o(a) LOCATÁRIO(A) proibido(a) de sublocá-lo ou usá-lo de forma diferente do previsto, sem a devida autorização do PROPRIETÁRIO ou LOCADOR.
    <br/><br/>
    Quaisquer alterações dos termos aqui dispostos só serão consideradas parte integrante deste contrato se realizada pelas partes e de acordo com suas regras de utilização.
    """

def gerar_clausulas_aluguel(dados, modalidade_deposito, opcoes_cobranca):
    """
    Gera o texto da seção de Aluguel e Encargos, incluindo dinamicamente
    a taxa de manutenção, se aplicável.
    """
    if modalidade_deposito == '1': # À vista
        texto_pagamento_deposito = "O pagamento do valor integral do depósito será realizado no ato da assinatura deste contrato."
    else: # Parcelado
        texto_pagamento_deposito = """
        O pagamento será realizado da seguinte forma:
        <br/><br/>
        a) O valor correspondente a 1,5 (um e meio) aluguel será pago no ato da assinatura deste contrato, a título de primeira parcela da caução.
        <br/><br/>
        b) O valor restante de 0,5 (meio) aluguel será pago juntamente com a mensalidade do segundo mês de vigência do contrato, quitando o total da garantia.
        """

    texto_taxa_manutencao = ""
    if opcoes_cobranca.get('taxa_tipo') == '1':
        valor_taxa = opcoes_cobranca.get('taxa_valor', '[VALOR NÃO INFORMADO]')
        texto_taxa_manutencao = f"""
        <br/><br/>
        Adicionalmente, será cobrada uma taxa de manutenção das áreas comuns no valor fixo de <b>R$ {valor_taxa}</b>, a ser paga juntamente com o aluguel mensal.
        """

    return f"""
    O(A) locatário(a) pagará o valor de <b>{dados.get('VALOR_ALUGUEL', '[VALOR NÃO ENCONTRADO]')}</b> referente ao aluguel do imóvel, o qual deverá ser pago antecipadamente até o dia <b>{dados.get('DIA_PAGAMENTO', '[DIA NÃO ENCONTRADO]')}</b> de cada mês, referente ao mês vincendo.
    {texto_taxa_manutencao}
    <br/><br/>
    Será cobrada como garantia para retenção do imóvel um depósito caução no valor total de <b>{dados.get('VALOR_DEPOSITO', '[VALOR NÃO ENCONTRADO]')}</b>, equivalente a 2 (dois) meses de aluguel. {texto_pagamento_deposito}
    <br/><br/>
    O valor nominal do aluguel poderá sofrer reajuste anual, de acordo com o Índice Nacional de Preços ao Consumidor Amplo (IPCA). No caso de atraso no pagamento do aluguel, será cobrada multa de 10% (dez por cento) sobre o valor total devido.
    <br/><br/>
    A inadimplência do pagamento do aluguel por 25 (vinte e cinco) dias seguidos terá como consequência o cancelamento deste contrato e a desocupação imediata do imóvel, sem o benefício do ressarcimento do depósito.
    """

def numero_por_extenso(n_str):
    """Converte um número (string) em sua representação por extenso."""
    mapa = {
        '1': 'um', '2': 'dois', '3': 'três', '4': 'quatro', '5': 'cinco',
        '6': 'seis', '7': 'sete', '8': 'oito', '9': 'nove', '10': 'dez'
    }
    return mapa.get(n_str, n_str)

def gerar_clausulas_gerais(opcoes_cobranca, modalidade_vigencia, opcoes_multa):
    """
    Gera as cláusulas gerais, ajustando os textos de serviços públicos e
    rescisão contratual com base nas escolhas do usuário.
    """
    # --- GERAÇÃO DA CLÁUSULA DE SERVIÇOS PÚBLICOS ---
    if opcoes_cobranca.get('modalidade') == '1':
        # Constrói o texto de cobrança de água
        if opcoes_cobranca.get('agua_tipo') == '1':
            texto_agua = f"um valor fixo de R$ {opcoes_cobranca.get('agua_valor', '0,00')} referente à água e esgoto"
        else:
            texto_agua = f"um percentual de {opcoes_cobranca.get('agua_valor', '0')}% sobre o valor total da fatura de água e esgoto"

        # Constrói o texto de cobrança de energia
        if opcoes_cobranca.get('luz_tipo') == '1':
            texto_luz = f"um valor fixo de R$ {opcoes_cobranca.get('luz_valor', '0,00')} referente à energia elétrica"
        else:
            texto_luz = f"um percentual de {opcoes_cobranca.get('luz_valor', '0')}% sobre o valor total da fatura de energia elétrica"

        texto_utilidade_publica = f"""
        Os valores de serviços de utilidade pública serão cobrados da seguinte forma: o(a) LOCATÁRIO(A) pagará à LOCADORA {texto_agua}, e {texto_luz}. Não haverá cobrança referente a cota condominial.
        <br/><br/>
        Não será permitido nenhum uso de aparelho elétrico que contenha resistência. O descumprimento dessa cláusula acarretará à uma multa de 1 aluguel e/ou cancelamento do contrato.
        """
    else: # modalidade_cobranca == '2'
        texto_utilidade_publica = """
        Não haverá cobrança de taxa referente à cota de consumo de água ou energia elétrica por parte da LOCADORA, estando estes itens inclusos no valor do aluguel.
        <br/><br/>
        Não será permitido nenhum uso de aparelho elétrico que contenha resistência. O descumprimento dessa cláusula acarretará à uma multa de 1 aluguel e/ou cancelamento do contrato.
        """

    # --- GERAÇÃO DA CLÁUSULA DE RESCISÃO ---
    texto_clausula_multa = ""
    if opcoes_multa.get('tipo') == '2': # Multa Fixa
        meses = opcoes_multa.get('valor_meses', '1')
        meses_str = numero_por_extenso(meses)
        plural_aluguel = "aluguéis" if int(meses) > 1 else "aluguel"
        texto_clausula_multa = f"incidirá multa contratual no valor fixo de {meses} ({meses_str}) {plural_aluguel} vigente(s)"
    else: # Multa Proporcional (Padrão)
         texto_clausula_multa = "incidirá multa contratual no valor equivalente a 1 (um) mês de aluguel vigente, calculada de forma proporcional"

    if modalidade_vigencia == '1': # Típico
        texto_rescisao = f"""
        Ao(À) LOCATÁRIO(A) é facultada a rescisão unilateral do contrato após transcorridos 12 (doze) meses de vigência, ficando isento(a) da cobrança de multa, desde que realize a notificação por escrito à LOCADORA com antecedência mínima de 30 (trinta) dias. Caso o(a) LOCATÁRIO(A) opte pela rescisão antes de completado o prazo de 12 (doze) meses, contados da data de assinatura, {texto_clausula_multa}, a qual poderá ser descontada do depósito de garantia.
        """
    else: # Atípico
        if opcoes_multa.get('tipo') == '1': # Se for atípico e proporcional, ajusta o texto
            texto_clausula_multa += " ao período restante do contrato"

        texto_rescisao = f"""
        Caso o(a) LOCATÁRIO(A) opte pela rescisão unilateral do contrato antes do término do prazo de vigência estipulado, {texto_clausula_multa}, a qual poderá ser descontada do depósito de garantia. A notificação de rescisão deverá ser feita por escrito à LOCADORA com antecedência mínima de 30 (trinta) dias.
        """

    return f"""
    <b>BENFEITORIAS E VISTORIAS</b>
    <br/><br/>
    Serão feitas vistorias de entrada e saída para registrar as condições do imóvel no início e no fim da locação. As condições do imóvel na entrada estão detalhadas no **Anexo I - Vistoria de Entrada**, o qual é parte integrante deste contrato e aceito pelo(a) LOCATÁRIO(A) nas condições ali descritas, sem ressalvas. Caso uma parte encontre qualquer divergência, deverá apontá-la e comprová-la com fotos, no prazo de 5 (cinco) dias do recebimento do laudo de vistoria, sob pena de aceitar integralmente a vistoria apresentada. A modificação ou benfeitoria no imóvel, quando que necessário ou útil, não poderá ser feita pelo(a) locatário(a) sem a prévia e expressa autorização do locador, por escrito, com detalhamento de todo o processo. A realização de qualquer benfeitoria não gerará reembolso ou qualquer retenção. O(A) locatário(a) deverá reparar prontamente os danos causados ao imóvel, às suas instalações, desde que ocasionado pelo(a) locatário(a), demais moradores, familiares ou demais visitantes, independentemente de culpa. O locador terá responsabilidade de reparar danos aparentes do imóvel anteriores ao início da locação quando afetarem a habitabilidade do imóvel.
    <br/><br/>
    <b>ENTREGA DAS CHAVES</b>
    <br/><br/>
    O(A) LOCATÁRIO(A) terá o prazo de 5 (cinco) dias corridos, a contar da data de assinatura do presente contrato, para realizar a retirada das chaves do imóvel. A não retirada das chaves dentro do prazo estipulado implicará na anulação automática do contrato, sem qualquer ônus para a LOCADORA.
    <br/><br/>
    <b>TITULARIDADE E SERVIÇOS DE UTILIDADE PÚBLICA</b>
    <br/><br/>
    {texto_utilidade_publica.strip()}
    <br/><br/>
    <b>RESCISÃO E DESCUMPRIMENTO DO CONTRATO</b>
    <br/><br/>
    {texto_rescisao.strip()}
    <br/><br/>
    <b>REGRAS DE CONVIVÊNCIA</b>
    <br/><br/>
    O(A) LOCATÁRIO(A) declara-se ciente e concorda em obedecer às regras de convivência do imóvel, comuns a todos os moradores, as quais foram informadas e aceitas no ato da assinatura deste contrato. O descumprimento de tais regras poderá implicar nas sanções previstas no regimento interno ou, em casos graves, na rescisão do presente contrato.
    <br/><br/>
    <b>DEVOLUÇÃO DO IMÓVEL</b>
    <br/><br/>
    Ao término da locação, o(a) locatário(a) deverá restituir ao locador o imóvel livre de pessoas e bens, e nas mesmas condições que o alugou e que foram registradas na vistoria de entrada. O(A) locatário(a) deverá notificar o locador previamente para realização de vistoria de saída, com objetivo de comprovar a desocupação do imóvel nas mesmas condições em que o recebeu. Caso o(a) locatário(a) não devolva o imóvel nas mesmas condições em que o recebeu, o(a) mesmo(a) terá o prazo adicional de 7 (sete) dias, contados do recebimento da notificação nesse sentido, para fazer todos os reparos necessários. Caso o(a) locatário(a) não faça a integralidade dos reparos nesse prazo, o locador fará os reparos, podendo ser descontado do valor dado como depósito. Tendo em vista que o(a) locatário(a) apresentou garantia em espécie no valor de 1 (um) aluguel, esse valor será devolvido ao final do contrato, descontando as devidas reformas e reparos necessários, com apresentação de notas fiscais de materiais e mão-de-obra.
    <br/><br/>
    <b>DISPOSIÇÕES FINAIS</b>
    <br/><br/>
    As partes estão cientes e de acordo que devem manter cadastro atualizado (telefone, e-mail, etc); Caso haja mudança de meios de contato sem a devida atualização, todas as comunicações enviadas para os contatos que as partes forneceram serão consideradas válidas para todos os efeitos legais. Assinatura Eletrônica: As Partes acordam que o envio de páginas de assinaturas no formato PDF e/ou as assinaturas eletrônicas do presente Contrato, por meio de plataformas de assinatura eletrônica, devidamente autorizadas pela Infraestrutura de Chaves Públicas Brasileira - ICP-Brasil (e.g., Docusign), serão consideradas válidas, sendo o Contrato considerado executável, válido e vigente entre as Partes.
    """

def gerar_secao_final(dados):
    """Gera a seção final para data e assinaturas."""
    return f"""
    DECLARANDO AMBAS AS PARTES DE ACORDO COM TODO O CONTEXTO DESSE CONTRATO, DATAM E ASSINAM.
    <br/><br/><br/>
    {dados.get('CIDADE_ASSINATURA', '[CIDADE NÃO ENCONTRADO]')}, {dados.get('DATA_ASSINATURA', '[DATA NÃO ENCONTRADO]')}.
    <br/><br/><br/><br/>
    _________________________________________
    <br/>
    {dados.get('LOCADORA_NOME', '[NOME NÃO ENCONTRADO]')}
    <br/>
    LOCADORA
    <br/><br/><br/>
    _________________________________________
    <br/>
    {dados.get('LOCATARIO_NOME', '[NOME NÃO ENCONTRADO]')}
    <br/>
    LOCATÁRIO(A)
    <br/><br/><br/>
    <b>Testemunhas:</b>
    <br/><br/><br/><br/>
    _________________________________________
    <br/>
    {dados.get('TESTEMUNHA1_NOME', '[NOME NÃO ENCONTRADO]')}
    <br/>
    CPF: {dados.get('TESTEMUNHA1_CPF', '[CPF NÃO ENCONTRADO]')}
    <br/><br/><br/>
    _________________________________________
    <br/>
    {dados.get('TESTEMUNHA2_NOME', '[NOME NÃO ENCONTRADO]')}
    <br/>
    CPF: {dados.get('TESTEMUNHA2_CPF', '[CPF NÃO ENCONTRADO]')}
    """

def gerar_anexo_vistoria(doc, imagens_dir, styles):
    """
    Gera o conteúdo do Anexo I, incluindo todas as imagens encontradas.
    """
    story_anexo = []

    story_anexo.append(PageBreak())
    story_anexo.append(Paragraph("ANEXO I - VISTORIA DE ENTRADA", styles['CenterBoldTitle']))
    story_anexo.append(Spacer(1, 1*cm))

    texto_aceite = """
    O(A) LOCATÁRIO(A) declara, neste ato, que vistoriou o imóvel objeto deste contrato de locação e o aceita nas condições em que se encontra, conforme registrado em fotos anexadas e descrições a seguir. O(A) LOCATÁRIO(A) se compromete a devolver o imóvel nas mesmas condições de conservação, ressalvados o desgaste natural pelo uso normal.
    """
    story_anexo.append(Paragraph(texto_aceite, styles['Justify']))
    story_anexo.append(Spacer(1, 0.5*cm))

    if not os.path.exists(imagens_dir) or not os.path.isdir(imagens_dir):
        msg_erro = f"<i>Atenção: O diretório de imagens '{imagens_dir}' não foi encontrado. Nenhuma foto será anexada.</i>"
        story_anexo.append(Paragraph(msg_erro, styles['Center']))
    else:
        fotos = sorted([f for f in os.listdir(imagens_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        if not fotos:
            story_anexo.append(Paragraph("<i>Nenhuma foto encontrada no diretório 'imagens'.</i>", styles['Center']))
        else:
            for i, foto_nome in enumerate(fotos):
                caminho_foto = os.path.join(imagens_dir, foto_nome)
                try:
                    max_width = doc.width
                    max_height = doc.height * 0.75
                    img = Image(caminho_foto, hAlign='CENTER')
                    img_width, img_height = img.drawWidth, img.drawHeight
                    aspect_ratio = img_height / float(img_width)
                    new_width = max_width
                    new_height = new_width * aspect_ratio
                    if new_height > max_height:
                        new_height = max_height
                        new_width = new_height / aspect_ratio
                    img.drawWidth = new_width
                    img.drawHeight = new_height
                    story_anexo.append(img)
                    story_anexo.append(Paragraph(f"<i>Foto {i+1}</i>", styles['Center']))
                    story_anexo.append(Spacer(1, 0.5*cm))
                except Exception as e:
                    story_anexo.append(Paragraph(f"<i>Erro ao carregar a foto '{foto_nome}': {e}</i>", styles['Center']))

    story_anexo.append(Spacer(1, 1*cm))
    story_anexo.append(Paragraph("<i>(Este anexo foi aceito e assinado digitalmente/eletronicamente junto ao contrato principal.)</i>", styles['Justify']))
    return story_anexo

# --- 3. CLASSE AUXILIAR PARA O RODAPÉ ---

class GestorDePagina:
    def __init__(self, dados):
        self.dados = dados

    def rodape(self, canvas, doc):
        canvas.saveState()
        canvas.setFont('Helvetica', 9)
        texto_rodape = f"Contrato de Locação - {self.dados.get('LOCATARIO_NOME', '')} - {self.dados.get('DATA_ASSINATURA', '')}"
        canvas.drawString(doc.leftMargin, doc.bottomMargin - 1*cm, texto_rodape)
        texto_pagina = f"Página {doc.page}"
        canvas.drawRightString(doc.width + doc.leftMargin, doc.bottomMargin - 1*cm, texto_pagina)
        canvas.restoreState()

# --- 4. FUNÇÃO PRINCIPAL PARA GERAR O PDF ---

def criar_pdf(dados, imagens_dir, opcoes_cobranca, modalidade_vigencia, modalidade_deposito, opcoes_multa, nome_arquivo_saida="contrato_locacao.pdf"):
    """
    Cria o documento PDF completo com base nos dados e nas escolhas do usuário.
    """
    doc = SimpleDocTemplate(nome_arquivo_saida, pagesize=A4,
                            leftMargin=2.5*cm, rightMargin=2.5*cm,
                            topMargin=2.5*cm, bottomMargin=2.5*cm)
    story = []

    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(name='Justify', alignment=TA_JUSTIFY, fontName='Helvetica', fontSize=10, leading=14))
    styles.add(ParagraphStyle(name='CenterBoldTitle', alignment=TA_CENTER, fontName='Helvetica-Bold', fontSize=12, leading=16))
    styles.add(ParagraphStyle(name='Center', alignment=TA_CENTER, fontName='Helvetica', fontSize=10, leading=14))
    styles.add(ParagraphStyle(name='ClauseTitle', fontName='Helvetica-Bold', fontSize=10, leading=14, spaceBefore=6))

    story.append(Paragraph("CONTRATO DE LOCAÇÃO RESIDENCIAL", styles['CenterBoldTitle']))
    story.append(Spacer(1, 1*cm))
    story.append(Paragraph(gerar_qualificacao_e_intro(dados, modalidade_vigencia), styles['Justify']))
    story.append(Spacer(1, 1*cm))
    story.append(Paragraph("ALUGUEL E ENCARGOS LOCATÍCIOS", styles['ClauseTitle']))
    story.append(Spacer(1, 0.3*cm))
    story.append(Paragraph(gerar_clausulas_aluguel(dados, modalidade_deposito, opcoes_cobranca), styles['Justify']))
    story.append(Spacer(1, 1*cm))
    story.append(Paragraph(gerar_clausulas_gerais(opcoes_cobranca, modalidade_vigencia, opcoes_multa), styles['Justify']))
    story.append(Spacer(1, 1*cm))
    story.append(Paragraph(gerar_secao_final(dados), styles['Center']))
    story.extend(gerar_anexo_vistoria(doc, imagens_dir, styles))

    gestor_pagina = GestorDePagina(dados)
    try:
        doc.build(story, onFirstPage=gestor_pagina.rodape, onLaterPages=gestor_pagina.rodape)
        print(f"\nPDF '{nome_arquivo_saida}' gerado com sucesso!")
    except Exception as e:
        print(f"\nOcorreu um erro ao gerar o PDF: {e}")

# --- 5. BLOCO DE EXECUÇÃO PRINCIPAL ---

if __name__ == "__main__":
    try:
        script_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        script_dir = os.getcwd()

    arquivo_dados_path = os.path.join(script_dir, "dados.txt")
    imagens_dir_path = os.path.join(script_dir, "imagens")

    opcoes_cobranca = {}
    opcoes_multa = {}

    # PERGUNTA 1: MODALIDADE DE COBRANÇA GERAL
    modalidade_geral = ''
    while modalidade_geral not in ['1', '2']:
        print("\n+------------------------------------------------------+")
        print("| Escolha a modalidade de cobrança de água e luz:      |")
        print("|                                                      |")
        print("| 1: Com cobrança de taxas (definir a seguir)          |")
        print("| 2: Sem cobrança de taxas (incluso no aluguel)        |")
        print("+------------------------------------------------------+")
        modalidade_geral = input("Digite 1 ou 2 e pressione Enter: ").strip()
        opcoes_cobranca['modalidade'] = modalidade_geral
        if modalidade_geral not in ['1', '2']:
            print("\n>>> Opção inválida. Por favor, digite apenas 1 ou 2. <<<\n")

    if modalidade_geral == '1':
        # DETALHAMENTO ÁGUA
        tipo_agua = ''
        while tipo_agua not in ['1', '2']:
            print("\n--- Cobrança de Água e Esgoto ---")
            print("1: Valor Fixo (R$)")
            print("2: Percentual (%) da fatura")
            tipo_agua = input("Escolha a opção (1 ou 2): ").strip()
            if tipo_agua not in ['1', '2']: print("Opção inválida.")
        opcoes_cobranca['agua_tipo'] = tipo_agua
        valor_agua = input(f"Digite o {'Valor Fixo (Ex: 60,00)' if tipo_agua == '1' else 'Percentual (Ex: 50)'}: ").strip()
        opcoes_cobranca['agua_valor'] = valor_agua

        # DETALHAMENTO LUZ
        tipo_luz = ''
        while tipo_luz not in ['1', '2']:
            print("\n--- Cobrança de Energia Elétrica ---")
            print("1: Valor Fixo (R$)")
            print("2: Percentual (%) da fatura")
            tipo_luz = input("Escolha a opção (1 ou 2): ").strip()
            if tipo_luz not in ['1', '2']: print("Opção inválida.")
        opcoes_cobranca['luz_tipo'] = tipo_luz
        valor_luz = input(f"Digite o {'Valor Fixo (Ex: 100,00)' if tipo_luz == '1' else 'Percentual (Ex: 50)'}: ").strip()
        opcoes_cobranca['luz_valor'] = valor_luz

    # PERGUNTA 2: TAXA DE MANUTENÇÃO
    taxa_manutencao = ''
    while taxa_manutencao not in ['1', '2']:
        print("\n+------------------------------------------------------+")
        print("| Haverá cobrança de taxa de manutenção de área comum? |")
        print("|                                                      |")
        print("| 1: Sim                                               |")
        print("| 2: Não                                               |")
        print("+------------------------------------------------------+")
        taxa_manutencao = input("Digite 1 ou 2 e pressione Enter: ").strip()
        opcoes_cobranca['taxa_tipo'] = taxa_manutencao
        if taxa_manutencao not in ['1', '2']:
             print("\n>>> Opção inválida. Por favor, digite apenas 1 ou 2. <<<\n")

    if taxa_manutencao == '1':
        valor_taxa = input("Digite o valor fixo da taxa de manutenção (Ex: 50,00): ").strip()
        opcoes_cobranca['taxa_valor'] = valor_taxa

    # PERGUNTA 3: MODALIDADE DE VIGÊNCIA
    modalidade_vigencia = ''
    while modalidade_vigencia not in ['1', '2']:
        print("\n+------------------------------------------------------+")
        print("| Escolha o tipo de vigência do contrato:              |")
        print("|                                                      |")
        print("| 1: Típico (30 meses, sem multa após 12 meses)        |")
        print("| 2: Atípico (Prazo menor, com multa proporcional)     |")
        print("+------------------------------------------------------+")
        modalidade_vigencia = input("Digite 1 ou 2 e pressione Enter: ").strip()
        if modalidade_vigencia not in ['1', '2']:
            print("\n>>> Opção inválida. Por favor, digite apenas 1 ou 2. <<<\n")

    # PERGUNTA 4: MODALIDADE DE PAGAMENTO DO DEPÓSITO
    modalidade_deposito = ''
    while modalidade_deposito not in ['1', '2']:
        print("\n+------------------------------------------------------+")
        print("| Escolha a forma de pagamento do depósito caução:     |")
        print("|                                                      |")
        print("| 1: Pagamento à vista (valor total na assinatura)     |")
        print("| 2: Pagamento parcelado (conforme cláusula padrão)    |")
        print("+------------------------------------------------------+")
        modalidade_deposito = input("Digite 1 ou 2 e pressione Enter: ").strip()
        if modalidade_deposito not in ['1', '2']:
            print("\n>>> Opção inválida. Por favor, digite apenas 1 ou 2. <<<\n")

    # PERGUNTA 5: TIPO DE MULTA POR RESCISÃO
    tipo_multa = ''
    while tipo_multa not in ['1', '2']:
        print("\n+------------------------------------------------------+")
        print("| Escolha o tipo de multa por rescisão antecipada:     |")
        print("|                                                      |")
        print("| 1: Multa Proporcional (padrão)                       |")
        print("| 2: Multa Fixa (valor de X meses de aluguel)          |")
        print("+------------------------------------------------------+")
        tipo_multa = input("Digite 1 ou 2 e pressione Enter: ").strip()
        opcoes_multa['tipo'] = tipo_multa
        if tipo_multa not in ['1', '2']:
            print("\n>>> Opção inválida. Por favor, digite apenas 1 ou 2. <<<\n")

    if tipo_multa == '2':
        valor_meses_multa = ''
        while not valor_meses_multa.isdigit() or int(valor_meses_multa) <= 0:
            valor_meses_multa = input("Digite o nº de meses de aluguel para a multa (Ex: 1, 2, 3): ").strip()
            if not valor_meses_multa.isdigit() or int(valor_meses_multa) <= 0:
                print("\n>>> Valor inválido. Por favor, digite um número inteiro positivo. <<<\n")
        opcoes_multa['valor_meses'] = valor_meses_multa

    dados_do_contrato = ler_dados_contrato(arquivo_dados_path)

    if dados_do_contrato:
        criar_pdf(dados_do_contrato, imagens_dir_path, opcoes_cobranca, modalidade_vigencia, modalidade_deposito, opcoes_multa)


+------------------------------------------------------+
| Escolha a modalidade de cobrança de água e luz:      |
|                                                      |
| 1: Com cobrança de taxas (definir a seguir)          |
| 2: Sem cobrança de taxas (incluso no aluguel)        |
+------------------------------------------------------+
Digite 1 ou 2 e pressione Enter: 2

+------------------------------------------------------+
| Haverá cobrança de taxa de manutenção de área comum? |
|                                                      |
| 1: Sim                                               |
| 2: Não                                               |
+------------------------------------------------------+
Digite 1 ou 2 e pressione Enter: 2

+------------------------------------------------------+
| Escolha o tipo de vigência do contrato:              |
|                                                      |
| 1: Típico (30 meses, sem multa após 12 meses)        |
| 2: Atípico (P